In [1]:
# from PIL import Image

# # Cargar la imagen
# img = Image.open('imagen_original.jpeg')

# # Definir la matriz de transformación de perspectiva
# M = [[1.5, 0.5, -100], [-0.5, 1.5, -100], [0, 0.002, 1]]

# # Obtener las dimensiones de la imagen original
# w, h = img.size

# # Crear una nueva imagen vacía del mismo tamaño que la imagen original
# img_transformada = Image.new('RGB', (w, h))

# # Recorrer cada píxel de la imagen transformada
# for x_out in range(w):
#     for y_out in range(h):
        
#         # Aplicar la matriz de transformación inversa al píxel actual
#         x_in = int((M[0][0]*x_out + M[0][1]*y_out + M[0][2]) / (M[2][0]*x_out + M[2][1]*y_out + M[2][2]))
#         y_in = int((M[1][0]*x_out + M[1][1]*y_out + M[1][2]) / (M[2][0]*x_out + M[2][1]*y_out + M[2][2]))
        
#         # Si el píxel transformado se encuentra dentro de los límites de la imagen original,
#         # copiar el color del píxel correspondiente en la imagen original a la imagen transformada
#         if x_in >= 0 and x_in < w and y_in >= 0 and y_in < h:
#             img_transformada.putpixel((x_out, y_out), img.getpixel((x_in, y_in)))
            
# # Guardar la imagen transformada
# img_transformada.save('imagen_transformada.jpg')


Para comenzar se requeire conocer las coordenadas de los puntos de la hoja blanca, con la finalidad de obtener los datos de entrada de la imagen 

In [15]:
import cv2
import numpy as np

def clics(event,x,y,flags,param):
	global puntos
	if event == cv2.EVENT_LBUTTONDOWN:
		cv2.circle(imagen,(x,y),5,(0,255,0),2)
		puntos.append([x,y])
		print(puntos)

def uniendo4puntos(puntos):
	cv2.line(imagen,tuple(puntos[0]),tuple(puntos[1]),(255,0,0),1)
	cv2.line(imagen,tuple(puntos[0]),tuple(puntos[2]),(255,0,0),1)
	cv2.line(imagen,tuple(puntos[2]),tuple(puntos[3]),(255,0,0),1)
	cv2.line(imagen,tuple(puntos[1]),tuple(puntos[3]),(255,0,0),1)

puntos = []
imagen = cv2.imread('imagen_original.jpeg')
aux = imagen.copy()
cv2.namedWindow('Imagen')
cv2.setMouseCallback('Imagen',clics)

while True:

	if len(puntos) == 4:
		uniendo4puntos(puntos)
		pts1 = np.float32([puntos])
		pts2 = np.float32([[0,0], [1024,0], [0,768], [1024,768]])

		M = cv2.getPerspectiveTransform(pts1,pts2)
		dst = cv2.warpPerspective(imagen, M, (480,300))

		cv2.imshow('dst', dst)
	cv2.imshow('Imagen',imagen)
	
	k = cv2.waitKey(1) & 0xFF
	if k == ord('n'):
		imagen = aux.copy()
		puntos = []
		
	elif k == 27:
		break

cv2.destroyAllWindows()

[[213, 450]]
[[213, 450], [469, 425]]
[[213, 450], [469, 425], [210, 644]]
[[213, 450], [469, 425], [210, 644], [480, 647]]


Se pretende obtener los valores de las constantes a mediante operaciones matriciales como la traspuesta, la inversa y la multiplicación de las matrices.
1. Obtención de la transpuesta
2. Obtención de la inevrsa de una matriz
3. Obtención de la multiplicación de dos matrices

In [3]:
#Se cargan las librearias 
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2

#Se carga el fondo que tiene el panel y la imagen que se va a proyectar
fondo = Image.open("fondoPerspectiva.jpg")
imagen = Image.open("fox.png")


In [4]:
puntosFondo = np.array([[0,0],[0,0],[0,0],[0,0]],np.int32)
puntosImagen = np.array([[0,0],[0,0],[0,0],[0,0]],np.int32)

imagenUL = [0,0]
imagenUR = [imagen.size[0],0]
imagenDL = [0,imagen.size[1]]
imagenDR = [imagen.size[0],imagen.size[1]]

fondoUL = [1235,85]
fondoUR = [1518,17]
fondoDL = [1610,361]
fondoDR = [1981,222]

puntosFondo[0] = fondoUL
puntosFondo[1] = fondoUR
puntosFondo[2] = fondoDL
puntosFondo[3] = fondoDR

puntosImagen[0] = imagenUL
puntosImagen[1] = imagenUR
puntosImagen[2] = imagenDL
puntosImagen[3] = imagenDR

In [5]:
resultado = np.array([fondoUL[0], fondoUR[0], fondoDR[0], fondoDL[0], fondoUL[1], fondoUR[1],fondoDR[1],fondoDL[1]])
print(resultado)

matrix = np.array([[ imagenUL[0], imagenUL[1],  1, 0, 0 ,0, -imagenUL[0]*fondoUL[0], -imagenUL[1]*fondoUL[0]],
                    [imagenUR[0], imagenUR[1],  1, 0, 0 ,0, -imagenUR[0]*fondoUR[0], -imagenUR[1]*fondoUR[0]],
                    [imagenDR[0], imagenDR[1],  1, 0, 0 ,0, -imagenDR[0]*fondoDR[0], -imagenDR[1]*fondoDR[0]],
                    [imagenDL[0], imagenDL[1],  1, 0, 0 ,0, -imagenDL[0]*fondoDL[0], -imagenDL[1]*fondoDL[0]],
                    [0, 0 , 0, imagenUL[0], imagenUL[1], 1, -imagenUL[0]*fondoUL[1], -imagenUL[1]*fondoUL[1]],
                    [0, 0 , 0, imagenUR[0], imagenUR[1], 1, -imagenUR[0]*fondoUR[1], -imagenUR[1]*fondoUR[1]],
                    [0, 0 , 0, imagenDR[0], imagenDR[1], 1, -imagenDR[0]*fondoDR[1], -imagenDR[1]*fondoDR[1]],
                    [0, 0 , 0, imagenDL[0], imagenDL[1], 1, -imagenDL[0]*fondoDL[1], -imagenDL[1]*fondoDL[1]]])


print(matrix)

[1235 1518 1981 1610   85   17  222  361]
[[      0       0       1       0       0       0       0       0]
 [    320       0       1       0       0       0 -485760       0]
 [    320     427       1       0       0       0 -633920 -845887]
 [      0     427       1       0       0       0       0 -687470]
 [      0       0       0       0       0       1       0       0]
 [      0       0       0     320       0       1   -5440       0]
 [      0       0       0     320     427       1  -71040  -94794]
 [      0       0       0       0     427       1       0 -154147]]


In [6]:
def transformada (matriz):

    filas = len(matriz)
    columnas = len(matriz[0])

    traspuesta = [[0 for j in range(filas)] for i in range(columnas)]

    for i in range(filas):
        for j in range(columnas):
            traspuesta[j][i] = matriz[i][j]
    
    # print("Matriz:")
    # for fila in matriz:
    #     print(fila)
    # print("Traspuesta:")
    # for fila in traspuesta:
    #     print(fila)
    
    return traspuesta

In [8]:
# #realizado por chat
# def inversa(matriz):
#     # Obtener las dimensiones de la matriz
#     n = len(matriz)

# # Crear una matriz identidad del mismo tamaño que la matriz original
#     identidad = [[0 if j != i else 1 for j in range(n)] for i in range(n)]

# # Combinar la matriz original con la matriz identidad
#     matriz_extendida = [fila_matriz + fila_identidad for fila_matriz, fila_identidad in zip(matriz, identidad)]

# # Eliminación gaussiana
#     for i in range(n-1):
#     # Dividir la fila actual por el elemento diagonal
#         pivote = matriz_extendida[i][i]
#         for j in range((2 * n)-1):
#             matriz_extendida[i][j] /= pivote
#     # Restar la fila actual multiplicada por el elemento debajo del pivote
#         for k in range(n):
#             if k != i:
#                 factor = matriz_extendida[k][i]
#                 for j in range(2 * n):
#                     matriz_extendida[k][j] -= factor * matriz_extendida[i][j]

# # Obtener la matriz inversa
#     inversa = [[matriz_extendida[i][j] for j in range(n, 2 * n)] for i in range(n)]

# # Imprimir la matriz original y su inversa
#     print("Matriz original:")
#     for fila in matriz:
#         print(fila)
#     print("Matriz inversa:")
#     for fila in inversa:
#      print(fila)

#     return inversa

# MatInversa = inversa(matrix)

In [9]:
def inversa(a, eps=1.0/(10**10)):
    """Retorna la matriz inversa de a utilizando el método de Gauss-Jordan"""
    n = len(a)
    b = [[float(i == j) for i in range(n)] for j in range(n)]
    for i in range(n):
        # se elige el mejor pivote
        max_element = abs(a[i][i])
        max_row = i
        for k in range(i+1, n):
            if abs(a[k][i]) > max_element:
                max_element = abs(a[k][i])
                max_row = k
        if abs(max_element) < eps:
            return None
        # Intercambia filas
        a[i], a[max_row] = a[max_row], a[i]
        b[i], b[max_row] = b[max_row], b[i]
        # reduce
        pivot = a[i][i]
        a[i] = [aij / pivot for aij in a[i]]
        b[i] = [bij / pivot for bij in b[i]]
        for j in range(n):
            if j != i:
                factor = a[j][i]
                a[j] = [aij - factor * a[i][ai] for ai, aij in enumerate(a[j])]
                b[j] = [bij - factor * b[i][bi] for bi, bij in enumerate(b[j])]

    return b

# matrizInversa = gauss_jordan(matrix)
# for fila in matrizInversa:
#     print(fila)

# print(matrizInversa)

# inversa = np.linalg.inv(matrix)
# print(inversa)

In [10]:
n = len(matrix)
# Crear una matriz identidad del mismo tamaño que la matriz original
identidad = [[0 if j != i else 1 for j in range(n)] for i in range(n)]

for fila in identidad:
    print(fila)


[1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 1]


In [12]:
def multiplicar_matrices(matriz1, matriz2):
    # Obtener las dimensiones de las matrices
    filas_matriz1 = len(matriz1)
    columnas_matriz1 = len(matriz1[0])
    filas_matriz2 = len(matriz2)
    columnas_matriz2 = len(matriz2[0])
    
    # Verificar si las matrices se pueden multiplicar
    if columnas_matriz1 != filas_matriz2:
        print("No se pueden multiplicar las matrices. La columna de la primera matriz no coincide con la fila de la segunda matriz.")
        return None
    
    # Crear una matriz vacía para almacenar el resultado de la multiplicación
    resultado = []
    for i in range(filas_matriz1):
        fila_resultado = []
        for j in range(columnas_matriz2):
            fila_resultado.append(0)
        resultado.append(fila_resultado)
    
    # Calcular el producto de las matrices
    for i in range(filas_matriz1):
        for j in range(columnas_matriz2):
            for k in range(filas_matriz2):
                resultado[i][j] += matriz1[i][k] * matriz2[k][j]
    
    return resultado

In [13]:
def multiplicar_matricesF(matriz1, matriz2):
    # Obtener las dimensiones de las matrices
    filas_matriz1 = len(matriz1)
    columnas_matriz1 = len(matriz1[0])
    filas_matriz2 = len(matriz2)
    columnas_matriz2 = 1
    
    # Verificar si las matrices se pueden multiplicar
    if columnas_matriz1 != filas_matriz2:
        print("No se pueden multiplicar las matrices. La columna de la primera matriz no coincide con la fila de la segunda matriz.")
        return None
    
    # Crear una matriz vacía para almacenar el resultado de la multiplicación
    resultado = []
    for i in range(filas_matriz1):
        fila_resultado = []
        for j in range(columnas_matriz2):
            fila_resultado.append(0)
        resultado.append(fila_resultado)
    
    # Calcular el producto de las matrices
    for i in range(filas_matriz1):
        for j in range(columnas_matriz2):
            for k in range(filas_matriz2):
                resultado[i][j] += matriz1[i][k] * matriz2[k][j]
    
    return resultado

a = np.matmul(np.matmul(np.linalg.inv(np.matmul(matrix.transpose(),matrix)),matrix.transpose()), resultado)

In [14]:
resultado = np.array([fondoUL[0], fondoUR[0], fondoDR[0], fondoDL[0], fondoUL[1], fondoUR[1],fondoDR[1],fondoDL[1]])
primeraP = multiplicar_matrices(matrix,transformada(matrix))
segundaP = multiplicar_matrices(inversa(primeraP),transformada(matrix))
final = multiplicar_matricesF(segundaP,resultado)

print(final)

C:\Users\nanga\AppData\Local\Temp\ipykernel_3868\3675675607.py:25: RuntimeWarning: overflow encountered in scalar multiply
  resultado[i][j] += matriz1[i][k] * matriz2[k][j]
C:\Users\nanga\AppData\Local\Temp\ipykernel_3868\3675675607.py:25: RuntimeWarning: overflow encountered in scalar add
  resultado[i][j] += matriz1[i][k] * matriz2[k][j]


IndexError: invalid index to scalar variable.